In [11]:
# Import all the required libraries
import os
from google.colab import files

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Data Collection and Preprocessing

In [12]:
# Upload the Telco Customer Churn dataset CSV from local Drive
upload_file = files.upload()
list_files = os.listdir()

Saving Telco_Customer_Churn.csv to Telco_Customer_Churn (1).csv


In [13]:
# Read the data from the CSV file into DataFrame
file_name = 'Telco_Customer_Churn.csv';
data = pd.read_csv(file_name)
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## Handling Missing Values

Let us check for any missing values (whitespaces) in our dataset.

In [14]:
# Regex to replace the whitespace with NaN
data = data.replace(r'^\s*$', np.nan, regex=True)

# Calculate the count of null values in each column
null_counts = data.isnull().sum()

# Filter and print columns with null values (count greater than 0)
print(null_counts[null_counts > 0])

TotalCharges    11
dtype: int64


We can see that `'TotalCharges'` is the only column that has missing values. We can calculate the missing values by using the `'tenure'`, `'MonthlyCharges'`, `'Contract'` columns.

In [15]:
# Get the indices of rows where 'TotalCharges' is null in the 'data' DataFrame
data[data['TotalCharges'].isnull()].index.tolist()

[488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754]

In [16]:
# Find indices where 'TotalCharges' is null
indices_with_null_total_charges = data[data['TotalCharges'].isnull()].index.tolist()

# Iterate through the indices
for idx in indices_with_null_total_charges:
    # Extract relevant information for the current index
    tenure = int(np.maximum(data['tenure'].loc[idx], 1))
    monthly_charges = data['MonthlyCharges'].loc[idx]

    # Determine the multiplier based on the contract type
    if data['Contract'].loc[idx] == 'Two year':
        contract_multiplier = 24
    elif data['Contract'].loc[idx] == 'One year':
        contract_multiplier = 12
    else:
        contract_multiplier = 1

    # Calculate and assign the 'TotalCharges' value
    data.loc[idx, 'TotalCharges'] = tenure * monthly_charges * contract_multiplier

# Display the total NaN values in each columns
data.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Lastly, let's check for any duplicate rows in the dataset.

In [17]:
# Count and print the number of duplicated rows in the DataFrame
print(f'Duplicate Rows: {data.duplicated().sum()}')

Duplicate Rows: 0


We will now convert numeric values (0 and 1) in the `'SeniorCitizen'` column to corresponding string labels ('No' and 'Yes') to make it consistent with rest of the binary values.

In [18]:
# Map numeric values in the 'SeniorCitizen' column to corresponding string labels
data['SeniorCitizen'] = data['SeniorCitizen'].map({0: "No", 1: "Yes"})
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
